In [ ]:
from datasets import load_dataset

dataset = load_dataset("deepmind/code_contests")

In [16]:
print(dataset['train'])

# dataset['train'].json()

Dataset({
    features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'],
    num_rows: 13328
})


In [18]:
import json
problems = []

In [21]:
for dataset_type in ['train', 'test', 'valid']:
    with open(f'data/code_contests_{dataset_type}.jsonl', 'w', encoding='utf-8') as write_file:
        for problem in dataset['valid']:
            write_file.write(json.dumps(problem) + '\n')

In [33]:
import jsonlines
import json
for dataset_type in ['train', 'test', 'valid']:
    with jsonlines.open(f'data/code_contests_{dataset_type}.jsonl') as f:
        for problem in f:
            for delete_elements in ['difficulty', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description']:
                problem.pop(delete_elements)
            solutions = []
            # print(problem['solutions'].keys())
            for solution in problem['solutions']['solution']:
                if '#include' in solution or 'java.' in solution: continue
                solutions.append(solution)
            # print(solutions)
            incorrect_solutions = []
            # print(problem['incorrect_solutions'].keys())
            for solution in problem['incorrect_solutions']['solution']:
                if '#include' in solution or 'java.' in solution: continue
                incorrect_solutions.append(solution)
                
            if len(solutions) == 0 or len(incorrect_solutions) == 0: continue
            problem['solutions'] = solutions
            problem['incorrect_solutions'] = incorrect_solutions
            
            with open(f'data/code_contests_{dataset_type}_python.jsonl', 'a', encoding='utf-8') as write_file:
                write_file.write(json.dumps(problem) + '\n')
        # break

In [17]:
import jsonlines
import json
import re

for dataset_type in ['train', 'test', 'valid']:
    with open(f'data/code_contests_{dataset_type}_python_with_spec1.jsonl', 'w', encoding='utf-8') as write_file:
        write_file.write('')

with open('data/train_input_spec_sample3.jsonl', 'w', encoding='utf-8') as write_file:
    write_file.write('')
    
constraints_start_token = '\nconstraints\n'
input_start_token = '\ninput\n'
input_finish_token = '\noutput\n'

sample_names = ['p02659', '71_A', 'p02658', 'p02761', '231_A', 'p02623', 'p02642', 'p02702', '158_A', '118_A', 'p02743', '282_A', 'p02983', 'p02936', 'p02684', 'p02756', 'p02696', '112_A', '236_A', '263_A', 'p02802', 'p03161', '50_A', 'p02780', '96_A', 'p03013', 'p02922', 'p03038', 'p02641', '266_A', '546_A', 'p03069', 'p02953', '281_A', '977_A', 'p03017', 'p02629', '791_A', 'p02572', 'p02742', 'p03053', 'p02891', '69_A', 'p02690', 'p02616', 'p03471', 'p02579', 'p02755', 'p03244', 'p02609', '110_A', '1399_A', '1409_A', '58_A', 'p03037', 'p02682', 'p02646', '1409_D', 'p02995', 'p02754', '705_A', 'p02912', 'p02689', '271_A', '131_A', 'p02900', 'p02647', '41_A', 'p02574', 'p02660', '677_A', 'p02971', 'p02585', 'p02972', 'p02693', 'p02678', 'p02990', 'p03074', 'p02580', 'p02838', 'p02719', 'p03000', '467_A', 'p02718', 'p02630', '785_A', 'p02713', '266_B', 'p02596', 'p02975', '133_A', 'p02791', '61_A', '443_A', 'p03078', 'p03042', 'p02677', 'p02925', '344_A', 'p03324', 'p03086', '1352_A', 'p02760', '136_A', '996_A', 'p02730', 'p02862', '1343_B', '1367_A', '25_A', 'p02803', 'p02707', 'p02948', 'p03854', 'p02700', 'p03044', 'p03565', '1335_A', '469_A', '318_A', 'p02787', 'p03282', 'p03160', '479_A', 'p03221', 'p02818', '1154_A', 'p02850', 'p02923', 'p02924', 'p02960', 'p03014', 'p03309', 'p03241', '486_A', 'p03386', 'p02691', 'p02899', '141_A', 'p02697', 'p02615', 'p03077', 'p03289', 'p02762', 'p02571', '1343_A', 'p02911', '1364_A', '228_A', 'p03455', '1374_A', 'p03006', 'p02844', '148_A', '1399_B', '1360_A', '1367_B', '1385_B', 'p03262', '1353_B', 'p02683', 'p02726', '581_A', '492_B', 'p03274', 'p03076', 'p02612', 'p03137', '1097_A', 'p02796', 'p02793', '144_A', '1374_B', 'p03457', '510_A', 'p03107', 'p02963', 'p03033', 'p02679', '268_A', 'p03075', '155_A', 'p03032', 'p02973', 'p02814', 'p02727', 'p02954', 'p02675', 'p02624', 'p03494', 'p02681', '758_A', 'p03252', '750_A', '230_A', 'p03043', 'p02837', 'p02712', '1353_C', '1385_A', 'p02686', 'p03861', '1095_A', '339_B', 'p02880', 'p03108', 'p03048', '1358_A', 'p03127', 'p04043', '1335_B', 'p02582', 'p02597', '1374_C', 'p03838', '1220_A', '1294_A', '732_A', 'p02833', 'p03472', 'p02958', '703_A', '1352_C', 'p03290', 'p03285', 'p03722', 'p02598', '427_A', 'p03607', 'p03208', 'p03721', 'p02882', 'p02771', 'p03081', 'p03128', 'p02711', 'p02767', '1283_A', 'p03329', '723_A', 'p03150', 'p03281', 'p02949', '584_A', 'p02695', 'p02599', '313_A', 'p03680', 'p02883', '1368_A', 'p02763', 'p02959', 'p02788', 'p02772', 'p02928', 'p02701', 'p03625', 'p03254', 'p02549', 'p02556', '1426_A', 'p02692', 'p03659', 'p02957', '1343_C', '472_A', 'p02842', 'p02897', 'p02797', 'p03007', 'p02843', 'p02747', '1324_B', 'p02753', 'p03353', '1337_B', 'p03103', '1373_B', 'p03846', '1234_A', 'p03251', 'p03361', '432_A', 'p02676', '935_A', '1303_A', 'p03739', 'p03476', 'p02729', 'p03964', '1397_A', 'p02766', 'p02832', '492_A', 'p02952', 'p03380', 'p03283', 'p02860', '1360_C', 'p03543', '1335_C', '509_A', 'p02601', '1281_A', 'p02831', 'p03633', '489_C', 'p02781', '466_A', '1363_A', 'p02664', 'p03545', '490_A', '1352_B', '1359_A', '1341_A', 'p02819', 'p02984', 'p03095', 'p02863', '749_A', 'p03041', '1400_A', 'p02588', '1372_A', 'p03351', 'p02918', '1433_B', '460_A', 'p02839', 'p02993', 'p02744', '978_B', '379_A', 'p02536', 'p02994', 'p02733', 'p03775', 'p03637', 'p02720', '540_A', '1354_A', '1234_B1', 'p02613', '1389_A', 'p03148', '431_A', 'p03209', 'p03062', 'p02557', 'p02644', 'p02688', '1409_B', 'p02603', '1223_A', '1196_A', 'p03194', '731_A', 'p03478', '1296_B', 'p03371', 'p02586', 'p03456', '1370_A', '270_A', 'p03263', '711_A', 'p02783', 'p03438', 'p03998', 'p02548', '1345_A', '1228_A', 'p03253', '1355_A', 'p03608', 'p02550', '1398_A', 'p02991', 'p03504', 'p02867', '1398_B', '1388_A', '656_A', 'p03377', 'p03433', 'p03699', 'p02553', 'p03486', '1369_A', 'p03632', 'p03166', 'p02879', '1350_A', 'p03679', '519_B', '327_A', '1358_B', 'p02570', 'p03112', '381_A', '1183_A', 'p03576', '1285_A', '489_B', 'p03239', 'p03045', 'p03059', '1328_C', 'p03363', '1359_B', '1324_A', '1326_A', '1284_A', '1373_A', '474_A', '1419_D1', '1342_B', '1335_D', 'p02951', 'p02716', '1146_A', 'p02705', 'p02917', '1328_B', 'p03723', 'p02889', 'p03644', 'p02901', 'p02854', 'p02706', '1249_A', '1354_C1', 'p02847', '1381_A1', 'p02736', '1337_A', '1003_A', 'p02759', 'p02396', 'p03109', 'p03295', '579_A', '1365_A', '1362_A', '230_B', '867_A', 'p02939', 'p03634', 'p02821', '1213_A', 'p03612', 'p02777', '577_A', 'p03141', 'p02717', 'p03815', '1405_A', '1295_A', '1167_A', '1345_B', '1206_A', '599_A', '1382_B', '1433_C', 'p02554', 'p03785', '1283_B', 'p02576', '1391_A', '1207_A', 'p03427', 'p02935', '1385_C', '1352_D', '1369_B', '1230_A', 'p02903', 'p03745', 'p03261', 'p02909', '556_A', 'p03050', '1334_A', '43_A', '520_B', '706_B', 'p03448', 'p02723', '1334_B', '1304_A', '151_A', '1108_A', '1326_B', '1395_A', '1391_B', '734_B', '1300_A', '1316_A', '1244_A', 'p04011', 'p02657', 'p02824', 'p02873', 'p03318', '1333_A', 'p03803', 'p02537', '894_A', '1272_A', 'p02805', '1141_A', 'p02578', 'p02648', '1180_A', '1324_C', 'p02970', 'p03212', '1144_A', 'p02708', 'p02916', 'p02685', '1401_B', 'p02812', 'p02577', 'p02687', 'p03323', 'p03284', 'p02919', 'p03806', 'p03031', 'p03200', 'p02846', 'p03805', '988_A', 'p03231', 'p02709', 'p02792', '1392_A', '1348_B', '1301_A', 'p02265', '1005_A', '978_A', '1325_A', '1015_A', '1406_A', '1372_B', '1380_A', 'p03162', 'p03229', '1354_B', '1392_B', '1399_C', 'p02390', '32_B', 'p03073', 'p02721', '1312_A', '977_C', '1141_B', 'p03067', '1330_A', 'p03008', '1339_B', '1420_A', '149_A', 'p02820', '1436_A', 'p02731', 'p02400', '835_A', '1419_A', 'p02933', '686_A', '82_A', 'p03434', '1360_E', '1294_B', '1426_B', '1371_C', '1332_A', '1371_B', '1176_A', 'p02921', 'p03994', '1409_C', '1323_A', '1351_B', '476_A', '459_B', '1270_A', '1145_A', '119_A', 'p03469', 'p04044', 'p02722', '1331_D', '1294_C', 'p02946', '34_B', 'p02258', 'p02926', '1111_A', 'p03089', 'p02988', 'p03776', '1375_A', 'p03767', 'p02618', 'p03943', 'p02786', '1405_B', '1430_B', 'p02811', '1006_A', 'p02835', 'p02848', '822_A', 'p03293', 'p02996', '999_A', 'p03986', '1080_A', 'p03145', 'p02834', '1114_A', '1291_A', 'p02829', 'p02749', 'p03169', '2_A', '1433_E', 'p03136', 'p03693', 'p02855', '1221_A', '987_A', '1133_A', '451_B', 'p02663', '1333_B', '742_A', 'p02394', 'p02981', 'p04030', 'p02698', '1370_C', '456_A', '1249_C1', 'p03243', '1428_A', '1393_A', 'p02989', 'p02547', 'p03215', 'p03730', 'p03593', '1091_A', '1430_A', 'p03011', '1249_B1', '1136_A', '1363_B', 'p03326', '1417_A', '1236_A', '1288_A', 'p02784', 'p03026', 'p02662', '1303_B', '320_A', '1293_B', 'p02594', 'p03408', '268_B', '1362_B', 'p02633', '1251_A', 'p03105', '1366_B', '1255_A', '1373_C', '1173_A', 'p03479', '1401_C', 'p03040', '1203_A', 'p03760', 'p03287', '1139_A', '1339_C', 'p02817', 'p03206', 'p02836', '1204_B', '1213_B', '1427_A', 'p02789', 'p03242', '984_A', 'p03131', 'p02904', '448_A', 'p03260', 'p02856', 'p03147', '1183_B', '1304_B', '1200_A', '1311_B', 'p02628', '567_A', 'p03129', 'p03210', 'p03250', '1181_B', 'p03286', 'p02595', '1271_A', 'p02885', '1017_A', '1257_B', '1278_A', '1064_A', '1061_A', 'p02397', '1144_B', '1406_B', '233_A', 'p03814', 'p03696', '1148_A', '1282_A', 'p02391', 'p04029', '721_A', 'p02724', 'p02651', '1031_A', '1093_A', '1065_A', 'p02607', 'p03592', '1165_B', 'p03264', 'p02403', '1243_A', '908_A', '939_A', '1230_B', '75_A', 'p02399', 'p03029', '1187_A', '1185_A', 'p02606', 'p02845', '1421_A', '1365_C', '478_A', 'p03556', '1307_A', '1253_A', 'p02807', '1102_A', '1436_B', 'p03435', '1392_C', 'p02417', 'p03437', '1199_B', '598_A', '363_B', 'p03657', 'p02741', 'p03937', '1101_A', '1355_B', 'p03951', 'p03219', 'p03362', '1300_B', '1358_C', '124_A', '746_B', '1157_A', '1369_C', '1225_A', 'p03327', '1113_A', 'p03385', '1143_A', '832_A', '1175_A', '474_B', 'p03291', '1395_B', '368_B', 'p02868', '1364_B', '994_A', 'p03360', '1248_B', '1374_E1', 'p02614', 'p02790', '1216_B', 'p03416', 'p02669', '255_A', '1426_C', '1359_C', 'p03096', '1362_C', 'p03761', 'p03544', 'p02874', '1375_B', '1263_A', '1176_B', 'p02934', 'p03780', '746_A', '507_B', 'p02583', 'p03060', '766_B', '1433_D', '1344_A', '1443_A', 'p02262', '1195_B', '1237_A', '1352_F', 'p02255', 'p02898', 'p03477', 'p04020', '467_B', '1244_B', 'p03672', 'p03997', '1352_G', '1197_A', '476_B', 'p03503', '1445_A', '621_A', '1029_A', 'p03142', '1166_A', '1096_A', 'p02555', '1266_A', 'p02604', 'p03583', '1313_A', 'p02801', 'p03674', '937_A', 'p04012', 'p03331', '978_C', '1093_B', '330_A', 'p03328', 'p03962', 'p03837', '768_A', '651_A', 'p03307', 'p04031', '37_A', '459_A', 'p03700', '1420_B', '1097_B', '1217_A', 'p03752', '1152_A', '1208_A', '1305_A', '615_A', '1315_A', '1437_B', 'p03665', '1327_B', '1165_A', '1324_D', '1144_C', '265_A', '478_B', '515_A', 'p02409', '1269_A', '798_A', '1380_B', 'p03651', 'p02745', '550_C', 'p03110', 'p03345', 'p03425', 'p03294', '1428_C', '1285_C', 'p03346', '1265_A', '733_A', '1372_C', 'p02401', '115_A', 'p03273', '1159_A', '1130_A', 'p03852', 'p03798', '1110_A', '1266_B', '1157_C1', '1029_B', '439_A', '1417_B', '1133_C', 'p03523', '709_A', '1138_A', 'p03720', '1099_A', '1355_D', 'p03705', 'p02257', 'p03054', '313_B', '1350_B', '770_A', '903_A', '950_A', '1270_B', 'p03125', 'p03719', 'p03101', '349_A', 'p02419', '378_A', '1407_B', 'p03325', '1248_A', '118_B', 'p03265', 'p03407', '1418_B', '912_A', '672_A', '1272_B', '501_A', 'p04000', '701_A', 'p03836', 'p02886', 'p03774', '877_A', '1400_C', 'p03470', '1201_A', 'p03369', 'p03354', 'p02976', '915_A', '764_A', '461_A', '1203_B', 'p03605', 'p03569', 'p02605', '1038_A', '898_A', '1073_A', 'p03447', '1203_C', '755_A', '1157_B', 'p03288', 'p02621', '1393_B', 'p02728', 'p02734', 'p02680', '1131_A', 'p02927', 'p03631', 'p03480', '1047_B', 'p03502', 'p03773', 'p02851', '513_A', '1399_D', 'p03575', '102_B', '938_A', '1422_B', 'p03547', '393_A', '1398_C', '466_C', 'p02271', '519_A', '1379_A', 'p03844', 'p03449', 'p02552', 'p04006', '1016_A', '1201_B', '1391_C', '841_A', '34_A', 'p03834', 'p03959', '1139_B', '1197_B', '897_A', '922_A', 'p04019', 'p02631', '1245_A', '1296_D', 'p02915', '1209_A', '1225_B1', '1326_D1', 'p03338', 'p03370', 'p02703', '1060_A', '1043_A', '1301_B', '1311_C', '837_A', '1163_A', '1328_D', 'p02622', '1296_C', '1312_C', '1396_A', '551_A', '1066_A', '1421_B', '1419_C', 'p03821', 'p03330', 'p02782', '1027_A', '1363_C', '463_B', '1076_A', '1328_F', 'p03186', 'p03151', '672_B', 'p03168', '49_A', '1197_C', 'p02969', 'p03003', '688_A', '1054_A', '675_A', 'p03546', '946_A', '976_A', '1257_C', '219_A', '552_B', 'p02964', '1117_B', '1342_C', '727_A', '570_B', '688_B', '1028_A', '1307_B', '1389_C', '870_A', '595_A', '761_A', '831_B', '1326_C', 'p03315', 'p03828', '450_B', '1195_A', '1220_B', '1194_A', '931_A', '887_A', 'p02798', '6_A', 'p03171', '433_B', '3_A', '1025_A', '892_A', '514_B', '796_A', '1176_C', '404_A', '1244_C', '982_A', 'p02407', '1353_D', '1150_A', '139_A', '659_A', 'p03364', '630_C', '1177_A', '1379_B', '1313_C1', '559_B', '1320_A', '1443_B', '1149_A', '129_A', '1140_A', '854_A', '1076_B', 'p03558', '1256_B', '1327_C', '719_A', '205_A', '699_C', '507_A', '580_C', 'p03777', 'p02235', '900_A', '1183_C', '1178_A', '272_A', 'p03337', 'p04014', '1013_A', '962_A', '1373_D', '1322_A', 'p02600', '1032_A', '1348_C', '1245_B', '535_B', '981_A', '224_A', '1003_B', '1200_B', 'p03304', '516_A', '1006_C', '275_A', '1189_B', '519_C', '1389_B', '258_A', '1151_A', '1099_B', '1316_B', '1023_A', 'p04034', '199_A', 'p00004', '1106_A', '1404_A', '1020_B', '1371_D', '456_B', '610_A', '805_B', '977_D', '522_A', '1205_A', '1141_C', '1242_A', '1030_C', 'p03341', '1009_A', '980_A', '1133_D', 'p03971', '989_A', '248_A', '1196_B', '807_A', '300_A', 'p02408', '1351_C', '864_A']

print(len(sample_names))

index = 1200
# exit()
names = []
for dataset_type in ['train', 'test', 'valid']:
    with open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl', 'w', encoding='utf-8') as write_file:
        write_file.write('')
    with jsonlines.open(f'data/code_contests_{dataset_type}_python.jsonl') as f:
        for problem in f:
            origin_name = problem['name']
            name = re.split(r'[. ]', origin_name)[0]
            description = '\n' + str(problem['description'])
            problem['description'] = {'description' : description.strip()}
            
            const_idx = description.lower().find(constraints_start_token)
            start_idx = description.lower().find(input_start_token)
            finish_idx = description.lower().find(input_finish_token)
            
            problem['name'] = {'origin': origin_name}
            problem['name']['name'] = name
            problem['name']['index'] = index
            index += 1
            
            if start_idx < 0 or finish_idx < 0: 
                spec = ''
                
            # print(name)
            elif const_idx >= 0:
                spec = description[const_idx:finish_idx].strip()
            else:
                # continue
                spec = description[start_idx:finish_idx].strip()

                pass
            # print(spec)
            # break
            
            if not spec: 
                names.append(name)
                print(name)
            
            problem['description']['spec'] = spec
            problem['description']['description'] = description[:finish_idx].strip()
            problem['description']['origin'] = description
            
            if name in sample_names:
                index -= 1
                problem['name']['index'] = sample_names.index(name)
                with open('data/train_input_spec_sample3.jsonl', 'a', encoding='utf-8') as write_file:
                    write_file.write(json.dumps(problem) + '\n')
                pass
            else:
                with open(f'data/code_contests_{dataset_type}_python_with_spec1.jsonl', 'a', encoding='utf-8') as write_file:
                    write_file.write(json.dumps(problem) + '\n')
print(names)

1200
p01931
p02472
p03932
p01905
p02473
p02037
p01932
p03751
p03978
p02476
952_D
p01933
p00473
p03935
952_B
p00472
p00559
p01901
p02038
p02477
p01907
p02475
p00488
p01906
p02474
p00474
p03931
p02266
1145_G
['p01931', 'p02472', 'p03932', 'p01905', 'p02473', 'p02037', 'p01932', 'p03751', 'p03978', 'p02476', '952_D', 'p01933', 'p00473', 'p03935', '952_B', 'p00472', 'p00559', 'p01901', 'p02038', 'p02477', 'p01907', 'p02475', 'p00488', 'p01906', 'p02474', 'p00474', 'p03931', 'p02266', '1145_G']


In [2]:
import jsonlines
import json

problems = ['']*1200


with jsonlines.open('data/train_input_spec_sample3.jsonl') as f:
    for problem in f:
        problems[problem['name']['index']] = problem
        

with open('data/train_input_spec_sample.jsonl', 'w', encoding='utf-8') as f:
    for p in problems:
        f.write(json.dumps(p) + '\n')

In [2]:
'''
dataset 확인용
'''

import jsonlines
import json
import re

dataset = {'train' : [], 'test': [], 'valid' : []}

for dataset_type in ['train', 'test', 'valid']:
    with jsonlines.open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl') as f:
        for problem in f:
            name = problem['name']['name']
            index = problem['name']['index']
            
            dataset[dataset_type].append(f'{name} - {index}')

In [4]:
for dataset_type in ['train', 'test', 'valid']:
    print(dataset[dataset_type][0])
    print(len(dataset[dataset_type]))
    print()

1037_E - 1200
6942

1575_A - 8142
115

1548_D1 - 8257
94



In [18]:
import jsonlines
import json
import re

dataset = {'train' : [], 'test': [], 'valid' : []}

with jsonlines.open('data/train_input_spec_sample.jsonl') as f:
    for problem in f:

        print(problem['name'])
        

        print(problem.keys())
        print(problem['description'].keys())
        print(problem['description']['description'])
        break

'''
for dataset_type in ['train', 'test', 'valid']:
    with jsonlines.open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl') as f:
        for problem in f:
            # name = problem['name']['name']
            # index = problem['name']['index']
            if problem['name']['name'] != '1296_D': continue
            
            print(problem)
            
            break
            dataset[dataset_type].append(f'{name} - {index}')
problem['description']['spec']
'''

{'origin': 'p02659 AtCoder Beginner Contest 169 - Multiplication 3', 'name': 'p02659', 'index': 0}
dict_keys(['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'solutions', 'incorrect_solutions', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'])
dict_keys(['description', 'spec', 'origin'])
Compute A \times B, truncate its fractional part, and print the result as an integer.

Constraints

* 0 \leq A \leq 10^{15}
* 0 \leq B < 10
* A is an integer.
* B is a number with two digits after the decimal point.

Input

Input is given from Standard Input in the following format:


A B


"\nfor dataset_type in ['train', 'test', 'valid']:\n    with jsonlines.open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl') as f:\n        for problem in f:\n            # name = problem['name']['name']\n            # index = problem['name']['index']\n            if problem['name']['name'] != '1296_D': continue\n            \n            print(problem)\n            \n            break\n            dataset[dataset_type].append(f'{name} - {index}')\nproblem['description']['spec']\n"

In [15]:
'''
dataset에 109꼴 확인
'''

import jsonlines
import json
import re


with open('test_results/powers.txt', 'w', encoding='utf-8') as write_file:
    write_file.write('')

dataset = {'train' : [], 'test': [], 'valid' : []}

dataset_name = ['train_input_spec_sample3', 'code_contests_train_python_with_spec1', 'code_contests_test_python_with_spec1', 'code_contests_valid_python_with_spec1',]
for dataset_type in dataset_name:
    # print(f'data/code_contests_{dataset_type}_python_with_spec.jsonl')
    
    # continue
    # with jsonlines.open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl') as f:
    with jsonlines.open(f'data/{dataset_type}.jsonl') as f:
        for problem in f:
            name = problem['name']['name']
            index = problem['name']['index']
            description = problem['description']['spec']

            a = list(re.findall('10[1-9]+',description))
            a.extend(list(re.findall('1010', description)))
            if a:
                # print(description)
                # print(problem['name']['origin'])
                # print(f'{index} - {name}')
                with open('test_results/powers.txt', 'a', encoding='utf-8') as write_file:
                    problem_name = problem['name']['origin']
                    write_file.write(f'{dataset_type}\n{problem_name}\n{a}\n\n')
                    # write_file.write(problem['name']['origin'] + ' ' + a + '\n')
            # print(problem['description']['description'])
        
            

In [4]:
import jsonlines
import json
import re

'''
거듭제곱에서 ^기호가 없는 경우
^기호를 추가함
'''

dataset_name = ['train_input_spec_sample3', 'code_contests_train_python_with_spec1',]
dataset_name_new = ['train_input_spec_sample4', 'code_contests_train_python_with_spec2',]
for dataset_type, dataset_type_new in zip(dataset_name, dataset_name_new):
    # print(f'data/code_contests_{dataset_type}_python_with_spec.jsonl')
    
    # continue
    # with jsonlines.open(f'data/code_contests_{dataset_type}_python_with_spec.jsonl') as f:
    with jsonlines.open(f'data/{dataset_type}.jsonl') as f:
        with open(f'data/{dataset_type_new}.jsonl', 'w', encoding='utf-8') as write_file:
            for problem in f:
                # print(problem.keys())
                # break
                name = problem['name']['name']
                index = problem['name']['index']
                description = problem['description']['spec']
                print(name, index)
                
                if name in ['p01131', 'p00128', 'p00130', 'p00242', 'p01228']: continue

                a = list(re.findall('10[1-9]+',description))
                a.extend(list(re.findall('1010', description)))
                if problem['name']['name'] not in ['p00790', '409_I', '1505_D', '1341_D', '1340_B', '1107_D']:
                    for source in a:
                        target = source[:2]
                        target += '^' + source[2:]
                        problem['description']['description'] = problem['description']['description'].replace(source, target, 1)
                        problem['description']['spec'] = problem['description']['spec'].replace(source, target, 1)
                        problem['description']['origin'] = problem['description']['origin'].replace(source, target, 1)
                        
                        if 'spec' in problem:
                            problem['spec']['spec'] = problem['description']['spac']
                    
                write_file.write(json.dumps(problem, ensure_ascii=False) + '\n')
                
                        # write_file.write(problem['name']['origin'] + ' ' + a + '\n')
                # print(problem['description']['description'])
            
                

1060_A 1037
1101_A 778
1374_E1 805
1398_A 377
1421_B 1050
379_A 333
519_B 395
615_A 884
915_A 970
p02613 343
p02744 331
p02879 392
p03013 25
p03455 149
p03774 962
p03943 610
p02408 1197
1043_A 1038
1236_A 666
219_A 1075
268_A 179
478_A 761
p02781 307
p02916 520
p03194 356
p03503 847
p03986 622
1208_A 882
1311_C 1040
1334_A 483
1374_B 172
p02741 776
p03150 240
p03294 905
p03612 451
p02257 933
1144_B 728
1165_A 889
1244_C 1110
1285_C 907
1328_F 1059
1391_C 1017
931_A 1097
p02574 68
p02705 422
p02834 625
p02971 71
p03107 175
p03254 257
p03407 952
1301_B 1039
1344_A 830
248_A 1193
507_B 825
835_A 571
p02548 370
p02679 178
p02807 765
p03371 360
1225_B1 1032
1353_C 198
1373_B 277
1396_A 1047
p02603 353
p02734 988
p02868 800
p03003 1067
p03287 690
1152_A 881
1230_A 476
1443_A 831
381_A 400
870_A 1086
p02630 84
p02761 3
p03031 530
p03318 504
p03637 338
1076_B 1129
1097_B 878
1203_A 688
1392_B 552
1433_C 464
300_A 1196
489_B 404
513_A 998
p02578 512
p02709 536
p02838 79
p02975 89
764_A 971
854_